[IPython Notebook](alternate_backends.ipynb) |  [Python Script](alternate_backends.py)

Advanced: Alternate Backends
============================

Setup
-----------------------------

As always, let's do imports and initialize a logger and a new Bundle.  See [Building a System](building_a_system.html) for more details.

In [1]:
import phoebe
from phoebe import u # units
import numpy as np
import matplotlib.pyplot as plt

logger = phoebe.logger()

b = phoebe.default_binary()

/usr/local/lib/python2.7/dist-packages/astropy/units/quantity.py:782: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  return super(Quantity, self).__eq__(other)


And we'll attach some dummy datasets.  See [Datasets](datasets.html) for more details.

In [2]:
b.add_dataset('orb', times=np.linspace(0,10,1000), dataset='orb01', component=['primary', 'secondary'])

<ParameterSet: 2 parameters | components: primary, secondary>

In [3]:
b.add_dataset('lc', times=np.linspace(0,10,1000), dataset='lc01')

<ParameterSet: 12 parameters | kinds: lc, lc_dep>

Available Backends
----------------------------

See the [Compute Tutorial](./compute) for details on adding compute options and using them to create synthetic models.


### PHOEBE 1.0 Legacy

For more details, see [Comparing PHOEBE 2.0 vs PHOEBE Legacy](../examples/legacy)

In [4]:
b.add_compute('legacy', compute='legacycompute')
print b['legacycompute']

ParameterSet: 11 parameters
  protomesh@legacycompute@com...: False
    pbmesh@legacycompute@compute: False
  morphology@legacycompute@co...: Detached binary
    cindex@legacycompute@compute: False
  refl_num@legacycompute@compute: 1
        ie@legacycompute@compute: False
   enabled@legacycompute@compute: True
  atm@primary@legacycompute@c...: kurucz
  atm@secondary@legacycompute...: kurucz
  gridsize@primary@legacycomp...: 40
  gridsize@secondary@legacyco...: 40


Using Alternate Backends
---------------------------------


### Adding Compute Options

Adding a set of compute options for an alternate backend is just as easy as for the PHOEBE backend.  Simply provide the function or name of the function in phoebe.parameters.compute that points to the parameters for that backend.

Here we'll add the default PHOEBE backend as well as photodynam by Josh Carter.  Note that in order to use an alternate backend, that backend must be installed on your machine.

In [5]:
b.add_compute('phoebe', compute='phoebebackend')

<ParameterSet: 22 parameters | datasets: lc01, orb01>

In [6]:
print b['phoebebackend']

ParameterSet: 22 parameters
  dynamics_method@phoebebacke...: keplerian
      ltte@phoebebackend@compute: False
  reflection_method@phoebebac...: wilson
  boosting_method@phoebebacke...: none
  protomesh@phoebebackend@com...: False
    pbmesh@phoebebackend@compute: False
  eclipse_method@phoebebacken...: native
  horizon_method@phoebebacken...: boolean
  enabled@lc01@phoebebackend@...: True
  enabled@orb01@phoebebackend...: True
  mesh_method@primary@phoebeb...: marching
  mesh_method@secondary@phoeb...: marching
  delta@primary@phoebebackend...: 0.1
  delta@secondary@phoebebacke...: 0.1
  maxpoints@primary@phoebebac...: 100000
  maxpoints@secondary@phoebeb...: 100000
  distortion_method@primary@p...: roche
  distortion_method@secondary...: roche
  atm@primary@phoebebackend@c...: ck2004
  atm@secondary@phoebebackend...: ck2004
  lc_method@phoebebackend@com...: numerical
  fti_method@phoebebackend@co...: None


In [7]:
b.add_compute('legacy', compute='legacybackend')

<ParameterSet: 11 parameters | components: primary, secondary>

In [8]:
print b['legacybackend']

ParameterSet: 11 parameters
  protomesh@legacybackend@com...: False
    pbmesh@legacybackend@compute: False
  morphology@legacybackend@co...: Detached binary
    cindex@legacybackend@compute: False
  refl_num@legacybackend@compute: 1
        ie@legacybackend@compute: False
  enabled@lc01@legacybackend@...: True
  atm@primary@legacybackend@c...: kurucz
  atm@secondary@legacybackend...: kurucz
  gridsize@primary@legacyback...: 40
  gridsize@secondary@legacyba...: 40


### Running Compute

Nothing changes for running compute - simply provide the compute tag for those options.  Do note, however, that not all backends support all dataset types.  And some will raise errors - for instance, legacy doesn't support ld_func=='interp' so we'll have to change it to a supported option manually.

In [10]:
b.set_value_all('ld_func', 'logarithmic')
b.run_compute('legacybackend', model='pdresults')

Wed, 21 Sep 2016 14:38 PARAMETERS   WARNING ld_func='interp' only supported by atm='ck2004'
Wed, 21 Sep 2016 14:38 PARAMETERS   WARNING ld_func='interp' only supported by atm='ck2004'
Wed, 21 Sep 2016 14:38 IO           WARNING rpole has no phoebe 1 corollary
Wed, 21 Sep 2016 14:38 IO           WARNING freq has no phoebe 1 corollary
Wed, 21 Sep 2016 14:38 IO           WARNING frac_lost_bol has no phoebe 1 corollary
Wed, 21 Sep 2016 14:38 IO           WARNING ld_func_bol has no phoebe 1 corollary
Wed, 21 Sep 2016 14:38 IO           WARNING mass has no phoebe 1 corollary
Wed, 21 Sep 2016 14:38 IO           WARNING rpole has no phoebe 1 corollary
Wed, 21 Sep 2016 14:38 IO           WARNING freq has no phoebe 1 corollary
Wed, 21 Sep 2016 14:38 IO           WARNING frac_lost_bol has no phoebe 1 corollary
Wed, 21 Sep 2016 14:38 IO           WARNING ld_func_bol has no phoebe 1 corollary
Wed, 21 Sep 2016 14:38 IO           WARNING mass has no phoebe 1 corollary
Wed, 21 Sep 2016 14:38 IO       

<ParameterSet: 2 parameters | qualifiers: fluxes, times>